In [1]:
import numpy as np
import deepthought, mne, os

mne.set_log_level('WARNING')
# mne.set_log_level('INFO')
verbose = False

from openmiir.constants import STIMULUS_IDS
from openmiir.metadata import load_stimuli_metadata, get_stimuli_version
from openmiir.events import get_event_id
from deepthought.util.fs_util import save

sfreq = 512  # no down-sampling

# select data to be extracted
# subjects = ['P01','P04','P05','P06','P07','P09','P11','P12','P13','P14']
subjects = ['P01','P04','P06','P07','P09','P11','P12','P13','P14']  # excluding P05

conditions = [1]
stimuli = STIMULUS_IDS

include_cue = False
use_mastoid_reference = False

data_root = '/srv/mlcog/datasets/OpenMIIR/'
mne_data_root = os.path.join(data_root, 'eeg', 'mne')
ica_data_root = os.path.join(data_root, 'eeg', 'preprocessing', 'ica')
target_path = 'data/OpenMIIR-Perception-512Hz.pklz'

ModuleNotFoundError: No module named 'cPickle'

In [2]:
cache = dict()

In [3]:
from openmiir.preprocessing import load_and_preprocess_raw
from openmiir.preprocessing.pipeline import load_raw_info

for subject in subjects:
    print subject
    if subject in cache: continue
    
    info = load_raw_info(subject, mne_data_root=mne_data_root)  # need to keep info with bad channels for later

    raw, trial_events = load_and_preprocess_raw(subject, 
                                            mne_data_root=mne_data_root,
                                            ica_data_root=ica_data_root,
                                            onsets='audio',
                                            interpolate_bad_channels=True, 
                                            reference_mastoids=use_mastoid_reference,
                                            l_freq=0.5,
                                            h_freq=30,
                                            sfreq=sfreq,
                                            ica_cleaning=True,
                                            verbose=None,
                                            n_jobs=4)
    cache[subject] = raw, trial_events, info

In [4]:
trials = []
metadata = []
for subject in subjects:    
    # load metadata
    meta_version = get_stimuli_version(subject)
    meta = load_stimuli_metadata(data_root, meta_version)
    print subject, meta_version, raw.info['bads']
    
    raw, trial_events, info = cache[subject]
    for stim_id in stimuli:
        if include_cue:
            trial_len = meta[stim_id]['length_with_cue']
        else:
            trial_len = meta[stim_id]['length_without_cue']

        for cond in conditions:
            event_id = get_event_id(stim_id, cond)
            
            # select EEG channels
            eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude=[])

            epochs = mne.Epochs(raw, events=trial_events, event_id=event_id, 
                    tmin=0, tmax=trial_len, 
                    proj=False, picks=eeg_picks, preload=True, verbose=False)
            
            
            # TEMPORARY FIX for update from mne-python 0.9 to 0.10
            # interpolate bad channels again as ICA will not be applied on them
            epochs.info['bads'] = info['bads']  # set bad channel flags for interpolation
            epochs.interpolate_bads()

            for i, trial in enumerate(epochs.get_data()):                                

                trials.append(np.asarray(trial, dtype=np.float32))

                trial_meta = dict()
                trial_meta['subject'] = subject
                if cond == 1:
                    trial_meta['trial_type'] = 'perception'
                else:
                    trial_meta['trial_type'] = 'imagination'
                trial_meta['trial_no'] = i
                trial_meta['condition'] = cond
                trial_meta['stimulus_id'] = stim_id

#                 print trial_meta
                metadata.append(trial_meta)                

target_path = 'data/OpenMIIR-Perception-512Hz.pklz'
print 'saving dataset to', target_path
print len(trials), len(metadata)
save(target_path, (trials, metadata), mkdirs=True)

In [5]:
min_len, max_len = 10000, 0
for trial in trials:
    min_len = min(min_len, trial.shape[-1])
    max_len = max(max_len, trial.shape[-1])
print min_len, max_len